In [77]:
import sys
sys.path.append('/home/nweiss/gdrive/Year 2/Summer - Duwamish/duwamish/code/')
import process_raw_f_and_b_data, add_dioxin_furans, join_to_screening_levels, join_to_sample_points, merge_all_results
import os
import glob
import pandas as pd
import numpy as np

folder_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Results_NEW"


# Initiate lookup tables paths
processed_folder = os.path.join(folder_path, "Processed")


In [78]:

# create mini reports
    
######### Tree Planting Event ###################
tree_planting_ids = ['GUF-1-S-1', 'GUF-1-S-2', 'GUF-1-S-3', 'GUF-1-S-4']
all_results = merge_all_results.main(processed_path = processed_folder)

tree_planting = []
for sample_id in tree_planting_ids:
    df = all_results[all_results['Sample ID'] == sample_id]
    tree_planting.append(df)

tree_planting = pd.concat(tree_planting)
chemical_groups = ['Dioxin Furans', 'PCB', 'RCRA8', 'TPH', 'PAH']
medium = ['Soil', 'Water']
tree_planting.to_excel(f"{processed_folder}/tree_planting_event.xlsx", index = False, sheet_name = 'RAW_DATA')
with pd.ExcelWriter(f"{processed_folder}/tree_planting_event.xlsx", mode = 'a', if_sheet_exists='replace') as writer: 
    tree_planting.to_excel(writer, sheet_name = 'RAW_DATA')

In [79]:
tree_planting['SL_exceeded'] = np.where(tree_planting['SL_exceeded'] == 'Y', 1, 0)

In [80]:
for c in tree_planting.columns:
    print(c)

DATE
Sample ID
Medium
Chemical Group
Chemical
Land Use
Target Receptor
Transport Pathway
Exposure Pathway
Screening Level
SL Unit
Source
Result Value
Result Value Units
SL_exceeded
SL_diff
stringent_ind


In [81]:
chemical_groups

['Dioxin Furans', 'PCB', 'RCRA8', 'TPH', 'PAH']

In [82]:
medium

['Soil', 'Water']

In [83]:
med = 'Soil'
group = 'RCRA8'


In [94]:
def make_pivot(df, med, group, name):
    df = df[(df['Chemical Group']==group)]
    df = df[df['Medium']==med]
    df['Screening Level'] = df['Screening Level'].astype(float).apply(lambda x: round(x, 2 - int(floor(log10(abs(x))))))

    df.drop_duplicates(inplace = True)
    df.sort_values(by = 'Sample ID')
    df.set_index(['Sample ID', 'DATE'], append=True, inplace = True)

    pivot_df = df.pivot(columns=['Chemical Group', 'Chemical','Source', 'Screening Level', 'SL Unit'], values='SL_exceeded')
    pivot_df = pivot_df.groupby(['Sample ID', 'DATE']).agg(max)
    pivot_df.replace({1:"Y", 0:"N"}, inplace = True)
    if len(pivot_df)>0:
        try:
            with pd.ExcelWriter(f"{processed_folder}/{name}_event.xlsx", mode = 'a', if_sheet_exists='replace') as writer: 
                pivot_df.to_excel(writer, sheet_name = f'PIVOT_{group}_{med}')
        except Exception as e:
            print(group, med)
            print(e)

In [95]:
make_pivot(df=tree_planting, med=med, group=group, name='tree_planting')

In [49]:
pivot_df.head(8)

Chemical Group                                    RCRA8        \
Chemical                                        Arsenic         
Source                    EPA Regional Screening Levels         
Screening Level                                    0.77   5.5   
SL Unit                                           mg/kg mg/kg   
     Sample ID DATE                                             
1457 GUF-1-S-1 2024-02-10                           1.0   NaN   
1458 GUF-1-S-2 2024-02-10                           1.0   NaN   
1459 GUF-1-S-3 2024-02-10                           1.0   NaN   
1460 GUF-1-S-4 2024-02-10                           1.0   NaN   
1462 GUF-1-S-1 2024-02-10                           NaN   0.0   
1463 GUF-1-S-2 2024-02-10                           NaN   0.0   
1464 GUF-1-S-3 2024-02-10                           NaN   0.0   
1465 GUF-1-S-4 2024-02-10                           NaN   0.0   

Chemical Group                                                             \
Chemical                                                                    
Source                    Washington Department of Ecology Cleanup Levels   
Screening Level                                                        20   
SL Unit                                                             mg/kg   
     Sample ID DATE                                                         
1457 GUF-1-S-1 2024-02-10                                             NaN   
1458 GUF-1-S-2 2024-02-10                                             NaN   
1459 GUF-1-S-3 2024-02-10                                             NaN   
1460 GUF-1-S-4 2024-02-10                                             NaN   
1462 GUF-1-S-1 2024-02-10                                             NaN   
1463 GUF-1-S-2 2024-02-10                                             NaN   
1464 GUF-1-S-3 2024-02-10                                             NaN   
1465 GUF-1-S-4 2024-02-10                                             NaN   

Chemical Group                                                              \
Chemical                                                                     
Source                    EPA Ecological Soil Screening Levels         NaN   
Screening Level                                             46   4.5     7   
SL Unit                                                  mg/kg mg/kg mg/kg   
     Sample ID DATE                                                          
1457 GUF-1-S-1 2024-02-10                                  NaN   NaN   NaN   
1458 GUF-1-S-2 2024-02-10                                  NaN   NaN   NaN   
1459 GUF-1-S-3 2024-02-10                                  NaN   NaN   NaN   
1460 GUF-1-S-4 2024-02-10                                  NaN   NaN   NaN   
1462 GUF-1-S-1 2024-02-10                                  NaN   NaN   NaN   
1463 GUF-1-S-2 2024-02-10                                  NaN   NaN   NaN   
1464 GUF-1-S-3 2024-02-10                                  NaN   NaN   NaN   
1465 GUF-1-S-4 2024-02-10                                  NaN   NaN   NaN   

Chemical Group                                                                 \
Chemical                                                                        
Source                    Preliminary Cleanup Levels (PCUL) Spreadsheet         
Screening Level                                      0.6666666666666666 4.672   
SL Unit                                                           mg/kg mg/kg   
     Sample ID DATE                                                             
1457 GUF-1-S-1 2024-02-10                                           NaN   NaN   
1458 GUF-1-S-2 2024-02-10                                           NaN   NaN   
1459 GUF-1-S-3 2024-02-10                                           NaN   NaN   
1460 GUF-1-S-4 2024-02-10                                           NaN   NaN   
1462 GUF-1-S-1 2024-02-10                                           NaN   NaN   
1463 GUF-1-S-2 2024-02-10 

In [57]:
pivot_df

Chemical Group                               RCRA8        \
Chemical                                   Arsenic         
Source               EPA Regional Screening Levels         
Screening Level                               0.77   5.5   
SL Unit                                      mg/kg mg/kg   
Sample ID DATE                                             
GUF-1-S-1 2024-02-10                             Y     N   
GUF-1-S-2 2024-02-10                             Y     N   
GUF-1-S-3 2024-02-10                             Y     N   
GUF-1-S-4 2024-02-10                             Y     N   

Chemical Group                                                        \
Chemical                                                               
Source               Washington Department of Ecology Cleanup Levels   
Screening Level                                                   20   
SL Unit                                                        mg/kg   
Sample ID DATE                                                         
GUF-1-S-1 2024-02-10                                               N   
GUF-1-S-2 2024-02-10                                               N   
GUF-1-S-3 2024-02-10                                               N   
GUF-1-S-4 2024-02-10                                               N   

Chemical Group                                                         \
Chemical                                                                
Source               EPA Ecological Soil Screening Levels         NaN   
Screening Level                                        46   4.5     7   
SL Unit                                             mg/kg mg/kg mg/kg   
Sample ID DATE                                                          
GUF-1-S-1 2024-02-10                                    N     N     N   
GUF-1-S-2 2024-02-10                                    N     N     N   
GUF-1-S-3 2024-02-10                                    N     N     N   
GUF-1-S-4 2024-02-10                                    N     Y     N   

Chemical Group                                                            \
Chemical                                                                   
Source               Preliminary Cleanup Levels (PCUL) Spreadsheet         
Screening Level                                 0.6666666666666666 4.672   
SL Unit                                                      mg/kg mg/kg   
Sample ID DATE                                                             
GUF-1-S-1 2024-02-10                                             Y     N   
GUF-1-S-2 2024-02-10                                             Y     N   
GUF-1-S-3 2024-02-10                                             Y     N   
GUF-1-S-4 2024-02-10                                             Y     Y   

Chemical Group                                               ...  \
Chemical                                                     ...   
Source                                                       ...   
Screening Level      129.35504885993484 0.23429333333333333  ...   
SL Unit                           mg/kg               mg/kg  ...   
Sample ID DATE                                               ...   
GUF-1-S-1 2024-02-10                  N                   Y  ...   
GUF-1-S-2 2024-02-10                  N                   Y  ...   
GUF-1-S-3 2024-02-10                  N                   Y  ...   
GUF-1-S-4 2024-02-10                  N                   Y  ...   

Chemical Group                                                      \
Chemical                                                    Silver   
Source               Preliminary Cleanup Levels (PCUL) Spreadsheet   
Screening Level                                                400   
SL Unit                                                      mg/kg   
Sample ID DATE                                                       
GUF-1-S-1 2024-02-10                                             N   
GUF-1-S-2 2024-